# Document Extraction with Azure AI Document Intelligence and Microsoft Phi-4 (Text Only)

This sample demonstrates how to extract structured data from any document using Azure AI Document Intelligence and Microsoft's Phi-4 model.

![Data Extraction](../../../../images/extraction-document-intelligence-phi.png)

This is achieved by the following process:

- Analyze a document using Azure AI Document Intelligence's `prebuilt-layout` model to extract the structure as Markdown.
- Construct a system prompt that defines the instruction for extracting structured data from documents.
- Construct a user prompt that includes specific extraction instruction for the type of document, the expected JSON schema, and the Markdown content of the document
- Use the chat completions API with the Phi-4 model to generate a structured output from the content.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document to Markdown format using Azure AI Document Intelligence.
- Use prompt engineering techniques to instruct Phi-4 to extract structured data from a type of document using example JSON structures.
- Use the analysis result from Azure AI Document Intelligence to determine the confidence of the extracted structured output.

## Useful Tips

- Combine this technique with a [page classification](../../classification/README.md) approach to reduce the number of pages to extract from to only those that match your criteria for extraction.

## Setup

### Import modules

This sample takes advantage of the following .NET dependencies:

- **Azure.AI.DocumentIntelligence** to interface with the Azure AI Document Intelligence API for analyzing documents.
- **Azure.AI.Inference** to interface with a deployed Phi-4 serverless endpoint in Azure AI Foundry.
- **Azure.Identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local components are also used:

- [**Invoice**](../../modules/samples/models/Invoice.csx) to provide the expected structured output JSON schema for invoice documents.
- [**OpenAIStructuredOutputsHelpers**](../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx) to generate structured outputs from the OpenAI API and provide parsing functions for the response output.
- [**AccuracyEvaluator**](../../modules/samples/evaluation/AccuracyEvaluator.csx) to evaluate the output of the classification process with expected results.
- [**DocumentIntelligenceConfidence**](../../modules/samples/confidence/DocumentIntelligenceConfidence.csx) to calculate the confidence of the classification process based on the analysis result from the Azure AI Document Intelligence API request.
- [**DocumentProcessingResult**](../../modules/samples/models/DocumentProcessingResult.csx) to store the results of the classification process as a file.
- [**AppSettings**](../../modules/samples/AppSettings.csx) to access environment variables from the `.env` file.

In [1]:
#r "nuget: Azure.Identity, 1.13.2"
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: Azure.AI.Inference, 1.0.0-beta.4"
#r "nuget: Azure.AI.DocumentIntelligence, 1.0.0"
#r "nuget: DotNetEnv, 3.1.1"

#!import ../../modules/samples/AppSettings.csx
#!import ../../modules/samples/helpers/AzureInferenceHelpers.csx
#!import ../../modules/samples/helpers/OpenAIStructuredOutputsHelpers.csx
#!import ../../modules/samples/helpers/StopwatchContext.csx
#!import ../../modules/samples/models/Invoice.csx
#!import ../../modules/samples/models/DocumentProcessingResult.csx
#!import ../../modules/samples/evaluation/AccuracyEvaluator.csx
#!import ../../modules/samples/confidence/DocumentIntelligenceConfidence.csx

using System;
using System.IO;
using System.Text.Json;
using Azure;
using Azure.Core;
using Azure.Identity;
using Azure.AI.Inference;
using Azure.AI.DocumentIntelligence;
using DotNetEnv;

Installed Packages Azure.AI.DocumentIntelligence, 1.0.0 Azure.AI.Inference, 1.0.0-beta.4 Azure.AI.OpenAI, 2.1.0 Azure.Identity, 1.13.2 DotNetEnv, 3.1.1

### Configure the Azure services

To use Azure AI Document Intelligence and Azure OpenAI, their SDKs are used to create a client instance using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [ ]:
string workingDir = Path.GetFullPath("../../../../");
AppSettings settings = new AppSettings(new Dictionary<string, string>(Env.Load(Path.Combine(workingDir, ".env"))));
string samplePath = Path.Combine(workingDir, "samples/dotnet/extraction/text/");
string sampleName = "document-extraction-phi";

DefaultAzureCredential credential = new DefaultAzureCredential(
    new DefaultAzureCredentialOptions { 
        ExcludeWorkloadIdentityCredential = true,
        ExcludeAzureDeveloperCliCredential = true,
        ExcludeEnvironmentCredential = true,
        ExcludeManagedIdentityCredential = true,
        ExcludeAzurePowerShellCredential = true,
        ExcludeSharedTokenCacheCredential = true,
        ExcludeInteractiveBrowserCredential = true
    }
);

ChatCompletionsClient inferenceClient = new ChatCompletionsClient(
    new Uri(settings.AzureAIPhiEndpoint),
    new AzureKeyCredential(settings.AzureAIPhiPrimaryKey)
);

var documentIntelligenceClient = new DocumentIntelligenceClient(
    new Uri(settings.AzureAIServicesEndpoint),
    credential
);

### Establish the expected output

To compare the accuracy of the extraction process, the expected output of the extraction process has been defined in the following code block based on the details of an [Invoice](../../../assets/invoices/invoice_1.pdf).

> **Note**: More invoice examples can be found in the [assets folder](../../../assets/invoices). These examples include the PDF file and an associated JSON metadata file that provides the expected structured output. You can add your own scenarios by following the same structure.

The expected output has been defined by a human evaluating the document.

In [3]:
string path = Path.Combine(workingDir, "samples/assets/invoices/");
string metadataFName = "invoice_1.json"; // Change this to the file you want to evaluate
string metadataFPath = Path.Combine(path, metadataFName);

Dictionary<string, JsonElement> data = JsonSerializer.Deserialize<Dictionary<string, JsonElement>>(File.ReadAllText(metadataFPath));

Invoice expected = data["0_expected"].Deserialize<Invoice>(
    new JsonSerializerOptions(JsonSerializerDefaults.Web)
    {
        PropertyNamingPolicy = JsonNamingPolicy.SnakeCaseLower
    }
);
string pdfFName = data["fname"].GetString();
string pdfFPath = Path.Combine(path, pdfFName);

AccuracyEvaluator<Invoice> invoiceEvaluator = new AccuracyEvaluator<Invoice>(matchKeys: new List<string>() { "ProductCode", "Description" });

## Extract data from the document

The following code block executes the data extraction process using Azure AI Document Intelligence and Microsoft's Phi-4 model.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use Azure AI Document Intelligence to analyze the structure of the document and convert it to Markdown format using the pre-built layout model.
3. Using a Phi-4 Serverless Endpoint model deployment in Azure AI Foundry and prompt engineering techniques, extract a structured data transfer object (DTO) from the content of the Markdown.

In [4]:
AnalyzeResult result;

StopwatchContext diSw;

using (diSw = new StopwatchContext())
{
    var pollerResult = await documentIntelligenceClient.AnalyzeDocumentAsync(
        WaitUntil.Completed,
        options: new AnalyzeDocumentOptions(
            modelId: "prebuilt-layout",
            bytesSource: BinaryData.FromBytes(File.ReadAllBytes(pdfFPath)))
        {
            OutputContentFormat = DocumentContentFormat.Markdown
        }
    );

    result = pollerResult.Value;
}

string markdown = result.Content;

In [5]:
string systemPrompt = "You are an AI assistant that extracts data from documents.";

In [6]:
List<ChatMessageContentItem> userContent = new List<ChatMessageContentItem>();

In [7]:
StringBuilder userTextPromptBuilder = new StringBuilder();
userTextPromptBuilder.AppendLine("Extract the data from this invoice.");
userTextPromptBuilder.AppendLine("- If a value is not present, provide null.");
userTextPromptBuilder.AppendLine("- Dates should be in the format YYYY-MM-DD.");
userTextPromptBuilder.AppendLine($"- Strictly use the following JSON schema: {typeof(Invoice).ModelJsonSchema()}");
userTextPromptBuilder.AppendLine("- ONLY return the JSON object. DO NOT return as a JSON markdown code block. DO NOT include any other detail in your response.");

string userTextPrompt = userTextPromptBuilder.ToString();

userContent.Add(new ChatMessageTextContentItem(userTextPrompt));
userContent.Add(new ChatMessageTextContentItem(markdown));

In [8]:
ParsedChatCompletions<Invoice> completion;

StopwatchContext phiSw;

using (phiSw = new StopwatchContext())
{
    completion = await inferenceClient
        .CompleteAsync(new ChatCompletionsOptions()
        {
            Messages = new List<ChatRequestMessage>()
            {
                new ChatRequestSystemMessage(systemPrompt),
                new ChatRequestUserMessage(userContent)
            },
            Temperature = 0.1f,
            NucleusSamplingFactor = 0.1f,
        });
}

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the data extraction process.

This includes:

- The Markdown representation of the document structure as determined by Azure AI Document Intelligence.
- The accuracy of the structured data extraction comparing the expected output with the output generated by Microsoft's Phi-4 model.
- The confidence score of the structured data extraction by comparing against the Azure AI Document Intelligence analysis.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the Phi-4 model.

### Understanding Accuracy vs Confidence

When using AI to extract structured data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the extraction process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [9]:
// Displays the output of the Azure AI Document Intelligence pre-built layout analysis in Markdown format.
display(markdown);

NEXGEN

Innovation drives progress


# INVOICE


<table>
<tr>
<th>Customer:</th>
<th colspan="2">Sharp Consulting</th>
<th colspan="2">Address:</th>
<th colspan="3">73 Regal Way, Leeds, LS1 5AB, UK</th>
</tr>
<tr>
<td>Delv. Date:</td>
<td>5/16/2024</td>
<td colspan="2">Invoice Number:</td>
<td colspan="2">3847193</td>
<td>Purchase Order:</td>
<td>15931</td>
</tr>
</table>


<table>
<tr>
<th>Item Code</th>
<th>Item Desc.</th>
<th>Unit Price</th>
<th>Quantity</th>
<th>Total</th>
</tr>
<tr>
<td>MA197</td>
<td>STRETCHWRAP ROLL</td>
<td>16.62</td>
<td>5</td>
<td>83.10</td>
</tr>
<tr>
<td>ST4086</td>
<td>BALLPOINT PEN MED.</td>
<td>2.49</td>
<td>10</td>
<td>24.90</td>
</tr>
<tr>
<td>JF9912413BF</td>
<td>BUBBLE FILM ROLL CL.</td>
<td>15.46</td>
<td>12</td>
<td>185.52</td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>


In [10]:
// Gets the parsed Invoice object from the completion response.
var invoice = completion.Parsed;

In [11]:
// Determines the accuracy of the extracted data against the expected values.
var accuracy = invoiceEvaluator.Evaluate(expected, invoice);

In [12]:
// Determines the confidence of the extracted data using the Document Intelligence response.
var confidence = DocumentIntelligenceConfidence<Invoice>.EvaluateConfidence(invoice, result);

In [13]:
// Gets the total execution time of the data extraction process.
var totalElapsed = diSw.Elapsed + phiSw.Elapsed;

// Gets the prompt tokens and completion tokens from the completion response.
var promptTokens = completion.Usage.PromptTokens;
var completionTokens = completion.Usage.CompletionTokens;

In [14]:
// Save the output of the data extraction result
var extractionResult = new DataProcessingResult<Invoice>(
    invoice,
    accuracy,
    confidence,
    promptTokens,
    completionTokens,
    totalElapsed
);

var extractionResultJson = JsonSerializer.Serialize(extractionResult, new JsonSerializerOptions { WriteIndented = true });
var extractionResultFPath = Path.Combine(samplePath, $"{sampleName}.{pdfFName}.json");

await File.WriteAllTextAsync(extractionResultFPath, extractionResultJson);

In [15]:
// Display the outputs of the extraction process.
var output = new
{
    Accuracy = $"{float.Parse(accuracy["overall"].ToString()) * 100:0.00}%",
    Confidence = $"{float.Parse(confidence["_overall"].ToString()) * 100:0.00}%",
    ExecutionTime = $"{totalElapsed.TotalSeconds:0.00} seconds",
    DocumentIntelligenceExecutionTime = $"{diSw.Elapsed.TotalSeconds:0.00} seconds",
    PhiExecutionTime = $"{phiSw.Elapsed.TotalSeconds:0.00} seconds",
    PromptTokens = promptTokens,
    CompletionTokens = completionTokens,
};

display(output);
display(confidence);

Accuracy,95.95%
Confidence,52.28%
ExecutionTime,22.21 seconds
DocumentIntelligenceExecutionTime,6.31 seconds
PhiExecutionTime,15.91 seconds
PromptTokens,3328
CompletionTokens,554


key type value CustomerName System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Single 0.994 lines System.Collections.Generic.List<System.ValueTuple<System.Collections.Generic.List<Azure.AI.DocumentIntelligence.DocumentWord>,System.Single>> index value 0 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.995) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord Content Sharp Polygon [ 1.8255, 2.0364, 2.2135, 2.0364, 2.2134, 2.185, 1.8255, 2.1829 ] Span Azure.AI.DocumentIntelligence.DocumentSpan Confidence 0.998 1 Azure.AI.DocumentIntelligence.DocumentWord Content Consulting Polygon [ 2.2526, 2.0363, 2.9848, 2.038, 2.9848, 2.1901, 2.2525, 2.1852 ] Span Azure.AI.DocumentIntelligence.DocumentSpan Confidence 0.995 Item2 0.995 1 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.994) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord Content Sharp Polygon [ 1.8195, 2.0358, 2.2084, 2.0354, 2.2082, 2.1878, 1.8195, 2.1858 ] Span Azure.AI.DocumentIntelligence.DocumentSpan Confidence 0.998 1 Azure.AI.DocumentIntelligence.DocumentWord Content Consulting Polygon [ 2.251, 2.0354, 2.9734, 2.0353, 2.9734, 2.1933, 2.2509, 2.1879 ] Span Azure.AI.DocumentIntelligence.DocumentSpan Confidence 0.994 Item2 0.994 value System.String Sharp Consulting CustomerTaxId System.Collections.Generic.Dictionary<System.String,System.Object> key type value confidence System.Single 0 lines System.Collections.Generic.List<System.ValueTuple<System.Collections.Generic.List<Azure.AI.DocumentIntelligence.DocumentWord>,System.Single>> (empty) value System.String CustomerAddress System.Collections.Generic.Dictionary<System.String,System.Object> key value Street key type value confidence System.Single 0.987 lines System.Collections.Generic.List<System.ValueTuple<System.Collections.Generic.List<Azure.AI.DocumentIntelligence.DocumentWord>,System.Single>> index value 0 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.987) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord 1 Azure.AI.DocumentIntelligence.DocumentWord 2 Azure.AI.DocumentIntelligence.DocumentWord 3 Azure.AI.DocumentIntelligence.DocumentWord 4 Azure.AI.DocumentIntelligence.DocumentWord 5 Azure.AI.DocumentIntelligence.DocumentWord 6 Azure.AI.DocumentIntelligence.DocumentWord Item2 0.987 1 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.987) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord 1 Azure.AI.DocumentIntelligence.DocumentWord 2 Azure.AI.DocumentIntelligence.DocumentWord 3 Azure.AI.DocumentIntelligence.DocumentWord 4 Azure.AI.DocumentIntelligence.DocumentWord 5 Azure.AI.DocumentIntelligence.DocumentWord 6 Azure.AI.DocumentIntelligence.DocumentWord Item2 0.987 value System.String 73 Regal Way City key type value confidence System.Single 0.987 lines System.Collections.Generic.List<System.ValueTuple<System.Collections.Generic.List<Azure.AI.DocumentIntelligence.DocumentWord>,System.Single>> index value 0 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.987) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord 1 Azure.AI.DocumentIntelligence.DocumentWord 2 Azure.AI.DocumentIntelligence.DocumentWord 3 Azure.AI.DocumentIntelligence.DocumentWord 4 Azure.AI.DocumentIntelligence.DocumentWord 5 Azure.AI.DocumentIntelligence.DocumentWord 6 Azure.AI.DocumentIntelligence.DocumentWord Item2 0.987 1 (System.Collections.Generic.List`1[Azure.AI.DocumentIntelligence.DocumentWord], 0.987) Item1 index value 0 Azure.AI.DocumentIntelligence.DocumentWord 1 Azure.AI.DocumentIntelligence.DocumentWord 2 Azure.AI.DocumentIntelligence.DocumentWord 3 Azure.AI.DocumentIntelligence.DocumentWord 4 Azure.AI.DocumentIntelligence.DocumentWord 5 Azure.AI.DocumentIntelligence.DocumentWord 6 Azure.AI.DocumentIntelligence.DocumentWord Item2 0.987 value System.String Leeds State key 